<a href="https://colab.research.google.com/github/dotimothy/DuckDetector/blob/model/TrainDuckDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦆 DuckDetector: A Project for Automatically Detecting Ducks 🦆
**Author:** [Timothy Do](https://timothydo.me)

## Setup

In [1]:
# Check if Environment is in Google CoLab:
IN_COLAB = True
try:
    import google.colab
except:
    IN_COLAB = False

In [5]:
if(IN_COLAB):
    !curl https://raw.githubusercontent.com/dotimothy/DuckDetector/model/requirements.txt -o ./requirements.txt
!pip install -r requirements.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1653  100  1653    0     0   8424      0 --:--:-- --:--:-- --:--:--  8433
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=662246 sha256=d67cb3b314f8b82a8d89bf387f1cf7952a2b9966afc98b1f293750d1bbc82a41
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [6]:
# Importing Libraries
import os
import torch
import cv2 as cv
import numpy as np
import cpuinfo
import psutil
import pycuda.driver as cuda
import ultralytics

# For Printing Purposes
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [9]:
# Environment Specifications
print(f'{color.BOLD}***** Environment Specifications *****{color.END}')
print(f"{color.BOLD}In CoLab{color.END}: {IN_COLAB}\n")

# Device to Develop PyTorch Models On
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device = torch.device('cuda' if use_cuda else ('mps' if use_mps else 'cpu'))
print(f'{color.BOLD}CPU:{color.END} {cpuinfo.get_cpu_info()["brand_raw"]}')
print(f'\t- {color.BOLD}# CPU Cores:{color.END} {psutil.cpu_count(logical=False)}')
print(f'\t- {color.BOLD}# CPU Threads:{color.END} {psutil.cpu_count(logical=True)}')
print(f'\t- {color.BOLD}Total System RAM:{color.END} {psutil.virtual_memory().total/(1024**3):.1f} GB ')
print(f'\n{color.BOLD}GPU:{color.END} {torch.cuda.get_device_name(device) if use_cuda else None}')
if(use_cuda):
    print(f'\t- {color.BOLD}Total GPU VRAM:{color.END} {torch.cuda.get_device_properties(device).total_memory/(1024**3):.1f} GB')

***** Environment Specifications *****
In CoLab: True

CPU: Intel(R) Xeon(R) CPU @ 2.00GHz
	- # CPU Cores: 1
	- # CPU Threads: 2
	- Total System RAM: 12.7 GB 

GPU: Tesla T4
	- Total GPU VRAM: 14.7 GB


## Dataset Preprocessing